<a href="https://colab.research.google.com/github/bardiaHSZD/Algorithms_DataStructure/blob/main/Dynamic_Mode_Decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment Installation**

In [43]:
!apt install octave
!apt-get update --fix-missing
!apt install octave

Reading package lists... Done
Building dependency tree       
Reading state information... Done
octave is already the newest version (4.2.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bio

# **Dynamic Mode Decomposition:** Test

In [56]:
%%writefile dmd_test_01.m

clear all;

%Quick Test of DMD with control.

%Data collection

A = [1.5 0;0 0.1];
x0 = [4;7];
K = [-1];
m = 20;
DataX = x0;
DataU = [0];

B = [1;0];

for j = 1 : m 
   
  DataX(:,j+1) = A * (DataX(:,j)) + B.*(K*DataX(:,j));
  DataU(:,j) = K .* DataX(1,j);
end

%Data matrices
X   = DataX(:,1:end-1);
Xp  = DataX(:,2:end);
Ups = DataU;

%SVD
[U,Sig,V] = svd(X,'econ');

thresh = 1e-10;
r = length(find(diag(Sig)>thresh));

U    = U(:,1:r);
Sig  = Sig(1:r,1:r);
V    = V(:,1:r);

%DMD

A_DMD  = Xp*V*inv(Sig)*U'

%DMDc - B is known 

A_DMDc = (Xp - B*Ups)*V*inv(Sig)*U'

% Extra
[W,D] = eig(A_DMDc);

Phi = Xp * V * inv(Sig) * U'*U * W

% DMD Spectra
t = linspace (0,m,m);
dt = t(2) - t(1);
lambda = diag(D);
omega = log(lambda)/dt;

% Compute DMD Solution
x0 = [4;7];
K = [-1];
m = 20;
DataXdmd = x0;
DataUdmd = [0];

B = [1.0;0.0];

for j = 1 : m   
  DataXdmd(:,j+1) = A_DMDc * (DataXdmd(:,j)) + B.*(K*DataXdmd(:,j));
  DataUdmd(:,j) = K .* DataXdmd(1,j);
end

%Data matrices
XDMD   = DataXdmd(:,1:end-1)
Ups = DataUdmd;

Overwriting dmd_test_01.m


In [58]:
!octave dmd_test_01.m # -W : no window system

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
A_DMD =

   5.0000e-01  -2.1751e-17
  -7.4096e-17   1.0000e-01

A_DMDc =

   1.5000e+00   2.2711e-16
  -7.4096e-17   1.0000e-01

Phi =

  -0.266675  -0.422947
  -0.084589   0.053335

XDMD =

 Columns 1 through 6:

   4.0000e+00   2.0000e+00   1.0000e+00   5.0000e-01   2.5000e-01   1.2500e-01
   7.0000e+00   7.0000e-01   7.0000e-02   7.0000e-03   7.0000e-04   7.0000e-05

 Columns 7 through 12:

   6.2500e-02   3.1250e-02   1.5625e-02   7.8125e-03   3.9063e-03   1.9531e-03
   7.0000e-06   7.0000e-07   7.0000e-08   7.0000e-09   7.0000e-10   7.0000e-11

 Columns 13 through 18:

   9.7656e-04   4.8828e-04   2.4414e-04   1.2207e-04   6.1035e-05   3.0518e-05
   7.0000e-12   7.0000e-13   7.0000e-14   7.0000e-15   6.9999e-16   6.9994e-17

 Columns 19 and 20:

   1.5259e-05   7.6294e-06
   6.9972e-18   6.9859e-19



# **Dynamic Mode Decomposition:** Algorithm

In [27]:
%%writefile dmd_full.m

A = [1.5 0;0 0.1];
x0 = [4;7];
K = [-1];
m = 20;
DataX = x0;
DataU = [0];

B = [1;0];

for j = 1 : m 
   
  DataX(:,j+1) = A * (DataX(:,j)) + B.*(K*DataX(:,j));
  DataU(:,j) = K .* DataX(1,j);
end

%Data matrices
X   = DataX(:,1:end-1);
Xp  = DataX(:,2:end);
Ups = DataU;
# --------------------------------------------------------
#X   = StateData(:,1:end-1);
#Xp  = StateData(:,2:end);
#Ups = InputData(:,1:end-1);

## =======================================================

Omega = [X;Ups];

[U,Sig,V] = svd(Omega,'econ');

thresh = 1e-10;
rtil = length(find(diag(Sig)>thresh));

Util    = U(:,1:rtil);
Sigtil  = Sig(1:rtil,1:rtil);
Vtil    = V(:,1:rtil); 

[U,Sig,V] = svd(Xp,'econ');

thresh = 1e-10;
r = length(find(diag(Sig)>thresh));

Uhat    = U(:,1:r); 
Sighat  = Sig(1:r,1:r);
Vbar    = V(:,1:r); 

n = size(X,1); 
q = size(Ups,1);
U_1 = Util(1:n,:);
U_2 = Util(n+q:n+q,:);

approxA = Uhat'*(Xp)*Vtil*inv(Sigtil)*U_1'*Uhat
approxB = Uhat'*(Xp)*Vtil*inv(Sigtil)*U_2'

[W,D] = eig(approxA);

Phi = Xp * Vtil * inv(Sigtil) * U_1'*Uhat * W

Overwriting dmd_full.m


In [28]:
!octave dmd_full.m # -W : no window system

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
approxA =

   0.239052   0.039018
   0.039018   0.110948

approxB =

   0.240703
   0.067541

Phi =

  -2.5000e-01   2.3699e-17
   2.8402e-17   1.0000e-01



In [32]:
%%writefile dmd_full.m

A = [0.239 0.039;0.039 0.11];
x0 = [4;7];
K = [-1];
m = 20;
DataX = x0;
DataU = [0];

B = [0.24;0.067];

for j = 1 : m 
   
  DataX(:,j+1) = A * (DataX(:,j)) + B.*(K*DataX(:,j));
  DataU(:,j) = K .* DataX(1,j);
end

%Data matrices
X   = DataX(:,1:end-1);
Xp  = DataX(:,2:end);
Ups = DataU;

Overwriting dmd_full.m


In [33]:
!octave dmd_full_test.m # -W : no window system

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
X =

 Columns 1 through 6:

   2.4000e-01  -1.8003e-01   1.3765e-01  -1.0496e-01   8.0059e-02  -6.1064e-02
   6.7000e-02   1.6730e-02  -5.1808e-03   4.7986e-03  -3.5654e-03   2.7301e-03

 Columns 7 through 12:

   4.6576e-02  -3.5525e-02   2.7097e-02  -2.0668e-02   1.5764e-02  -1.2024e-02
  -2.0812e-03   1.5875e-03  -1.2109e-03   9.2358e-04  -7.0445e-04   5.3732e-04

 Columns 13 through 18:

   9.1713e-03  -6.9953e-03   5.3356e-03  -4.0697e-03   3.1041e-03  -2.3677e-03
  -4.0983e-04   3.1260e-04  -2.3843e-04   1.8186e-04  -1.3871e-04   1.0580e-04

 Columns 19 and 20:

   1.8059e-03  -1.3775e-03
  -8.0701e-05   6.1554e-05

